In [ ]:
import numpy as np
import pandas as pd
from chisq import EclipseFit
import emcee
%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('lines', linewidth=2)

In [ ]:
class Plotter(EclipseFit):
    def __init__(self, system, **kwargs):
        EclipseFit.__init__(self, system, **kwargs)        

    def plot_residuals(self, x, star='A', phased=False, ecl_max=None):
        cmap = plt.cm.get_cmap('inferno')
        ecl_model, rv_model = self.get_residuals(x, tFin=ecl_max)
        if not ecl_max:
            ecl_max = self.ecl_data[star]['data_t'].max() + 50
        ecl_model[star] = ecl_model[star][ecl_model[star]['model_t'] < ecl_max]
        fig, ax = plt.subplots(2, figsize=(12, 8), gridspec_kw={'height_ratios':[3,1]},
                               sharex=True)
        T0, P = lsq_fit(ecl_model[star]['model_t'])
        Tp2 = x[6]
        if self.system == '5095':
            P2 = 235.9 # Highest peak in Fourier transform of ETVs of KIC509
        elif self.system == '782':
            P2 = x[5]

        ax[0].scatter(self.time_to_phase(ecl_model[star]['model_t'], Tp2, P2, phased),
                      86400*ecl_time_to_etv(ecl_model[star]['model_t'], P, T0),
                      c=ecl_model[star]['model_t'], cmap=cmap, marker='x', s=64)

        ax[0].errorbar(self.time_to_phase(self.ecl_data[star]['data_t'], Tp2, P2, phased),
                       86400*ecl_time_to_etv(self.ecl_data[star]['data_t'], P, T0), 
                       86400*self.ecl_data[star]['data_err'], linestyle='None', marker='o', 
                       markersize=6, color='0.5')

        ax[1].errorbar(self.time_to_phase(self.ecl_data[star]['data_t'], Tp2, P2, phased),
                       86400*ecl_model[star]['res'].dropna(),
                       86400*self.ecl_data[star]['data_err'], linestyle='None', 
                       marker='o', markersize=6, color='0.5')
        ax[1].axhline(0, linestyle='--', color='k')

        ax[1].set_xlabel('Time (BJD-2454900)')
        ax[0].set_ylabel('ETV (seconds)')
        plt.subplots_adjust(hspace=0)
        
    def cont_rv(self, x, rv_times):
        gamma = x[-1]
        rv = {}
        for i in self.rv_stars:
            rv[i] = pd.DataFrame(index=range(len(rv_times)), columns=['time', 'rv'], dtype=float)
        rvcount = 0
        sim = self.set_up_sim(x)
        while rvcount < len(rv_times):
            sim.integrate(rv_times[rvcount])
            for i in self.rv_stars:
                rv[i].at[rvcount, 'time'] = sim.t
                rv[i].at[rvcount, 'rv'] = -sim.particles[i].vz*1731.45683681 + gamma
            rvcount += 1
        return rv

    def rv_residuals(self, x, phased=True, extra_rvs=None):
        gamma = x[-1]
        ecl_model, rv_model = self.get_residuals(x)
        T0, P = lsq_fit(ecl_model['A']['data_t'])
        if phased:
            cont_rvs = self.cont_rv(x, np.linspace(T0, T0 + P, 100, endpoint=False))
        else:
            start = self.rv_data['A']['time'].min()
            stop = self.rv_data['A']['time'].max()
            if extra_rvs:
                start = min(start, extra_rvs['A']['time'].min())
                stop = max(stop, extra_rvs['A']['time'].max())
            start -= 5.; stop += 5.
            cont_rvs = self.cont_rv(x, np.linspace(start, stop, 1000))
        fig, ax = plt.subplots(2, figsize=(16,10), gridspec_kw={'height_ratios':[3,1]},
                               sharex=True)
        colors = {'A':'r', 'B':'k'}
        for i in self.rv_stars:
            ax[0].errorbar(self.time_to_phase(self.rv_data[i]['time'], T0, P, phased), self.rv_data[i]['rv'], 
                           self.rv_data[i]['rv_err'], linestyle='None', marker='o', c=colors[i])
            if extra_rvs:
                ax[0].errorbar(self.time_to_phase(extra_rvs[i]['time'], T0, P, phased), extra_rvs[i]['rv'], 
                               extra_rvs[i]['rv_err'], linestyle='None', marker='o', c=colors[i])
            ax[0].plot(self.time_to_phase(cont_rvs[i]['time'], T0, P, phased), cont_rvs[i]['rv'], colors[i])

            ax[1].errorbar(self.time_to_phase(self.rv_data[i]['time'], T0, P, phased), rv_model[i]['res'], 
                           self.rv_data[i]['rv_err'], linestyle='None', marker='o', color=colors[i])
        ax[0].axhline(gamma, linestyle='--', color='k')
        ax[1].axhline(0, linestyle='--', color='k')
        if phased:
            ax[1].set_xlabel('Orbital Phase from Primary Eclipse')
        else:
            ax[1].set_xlabel('Time (BJD-2454900)')
        ax[0].set_ylabel('Radial Velocity (km/s)')
        plt.subplots_adjust(hspace=0)
                       
    def time_to_phase(self, t, T0, P, phased=True):
        if not phased:
            return t
        #T0, P = lsq_fit(self.ecl_data['A']['data_t'])
        return np.remainder(t - T0, P)/P

    def impact_fit_quality(self, x):
        ecl_model, rv_model = self.get_residuals(x)
        dbdt, b0 = self.impact_regression(ecl_model)
        if not self.b:
            return
        print('Data:  {: 6.4} t + {:5.4}'.format(self.dbdt_data['A'], self.b0_data['A']))
        print('Model: {: 6.4} t + {:5.4}'.format(dbdt['A'], b0['A']))

def lsq_fit(ecl_time):
    df = ecl_time.dropna()
    A = np.vstack([np.ones(len(df)), df.index]).T
    return np.linalg.lstsq(A, df, rcond=None)[0]

def ecl_time_to_etv(ecl_time, P=None, T0=None):
    if P is None or T0 is None:
        T0, P = lsq_fit(ecl_time)
    return ecl_time - P*ecl_time.index.values - T0

In [ ]:
def nice_units(x):
    y = x.copy()
    # convert angles to degrees
    y[...,2] *= 180/np.pi
    y[...,4] *= 180/np.pi
    y[...,9] *= 180/np.pi
    y[...,10] *= 180/np.pi
    # convert solar masses to jupiter masses
    y[...,13] *= 1047.58
    return y

def pretty_print(x):
    e2 = np.sqrt(x[7]**2 + x[8]**2)
    omega2 = np.degrees(np.arctan2(x[8], x[7]))
    return \
    '''
             P (d)    Epoch (d)   i (deg)        e    ω (deg)    Ω (deg)
Binary   {:>9.8}    {:>9.7}   {:>7.5}  {:>7.3}   {:> 8.4}   {:> 8.4}
Planet   {:>9.5}    {:>9.5}   {:>7.4}  {:>7.3}   {:> 8.4}   {:> 8.3}
 
Masses:
M_A (Msolar) {:.4}
M_B (Msolar) {:.4}
M_p (Mjup)   {:.4}
k2           {:.4}
γ (km/s)     {:.4}   
    '''.format(x[0], x[1], np.degrees(x[2]), x[3], np.degrees(x[4]), 0.,
               x[5], x[6], np.degrees(x[9]), e2, omega2, np.degrees(x[10]),
               x[11], x[12], 1047.5*x[13], x[14], x[15])

In [ ]:
system = '5095'

In [ ]:
chain = np.load(system + '_chains_carmenes.npy')
prob = np.load(system + '_probs_carmenes.npy')
good_walkers = -2*prob[:, -1] < 300
print('Good walkers:', np.sum(good_walkers))
best_indx = prob.flatten()[np.nonzero(prob.flatten())].argmax()
print('Best chi-squared:', -2*prob.flatten()[best_indx])
x = chain.reshape(-1, 16)[best_indx].copy()
#x = np.insert(x, 2, np.radians(90))
print(pretty_print(x))

In [ ]:
fit = EclipseFit(system)
#x = [18.6109138,  66.8627328, np.radians(90),  0.538172592, 1.91748714,  
#     239.503827,  93.9899100,  0.0398027125, -0.0408160527,  1.52275477, -0.00779595908,  
#     1.19411368, 1.02500160,  4.31304120e-03,  0.0,  4.23715912]
ecl_model, rv_model = fit.get_residuals(x)
dbdt, b0 = fit.impact_regression(ecl_model)
print(b0, dbdt)
print(-2*fit.evaluate(x))
#plt.plot(ecl_model['A']['model_t'], b0['A'] + dbdt['A']*ecl_model['A']['model_t'])
#plt.plot(ecl_model['A']['model_t'], ecl_model['A']['model_b'])

In [ ]:
#import cProfile
#cProfile.run('fit.get_residuals(x)', sort='time')

In [ ]:
plot = Plotter(system)
plot.plot_residuals(x, 'A', phased=True)
try:
    plot.plot_residuals(x, 'B', phased=True)
except:
    pass
plot.rv_residuals(x, phased=True)
plot.impact_fit_quality(x)

In [ ]:
sim = plot.set_up_sim(x)
N = 1000
prec = np.zeros((N,3))
ts = np.linspace(0, 1600, N)
for i in range(N):
    sim.integrate(ts[i])
    prec[i,0] = ts[i]
    prec[i,1] = sim.particles[1].omega
plt.plot(prec[...,0], prec[...,1])

In [ ]:
# New RVs
new_rvs = {}
new_rvs['A'] = pd.DataFrame([[58199.1470708842, 116.39772, 4.0],
                             [58212.147214721 , 64.621752, 4.0],
                             [58229.1205147477, 54.468166, 4.0],
                             [58236.0789507604, 102.59536, 4.0],
                             [58246.110852669 , 36.560824, 4.0],
                             [58251.1009844909, 85.059876, 4.0],
                             [58260.1059522964, 94.600916, 4.0],
                             [58264.0851431191, 29.099431, 4.0],
                             [58293.0101433173, 115.91943, 4.0]], columns=['time', 'rv', 'rv_err'])
new_rvs['B'] = pd.DataFrame([[58199.1470708842, 46.260634, 4.0],
                             [58212.147214721 , 80.610640, 4.0],
                             [58229.1205147477, 102.96722, 4.0],
                             [58236.0789507604, 41.304625, 4.0],
                             [58246.110852669 , 122.36684, 4.0],
                             [58251.1009844909, 73.121648, 4.0],
                             [58260.1059522964, 51.297680, 4.0],
                             [58264.0851431191, 135.05121, 4.0],
                             [58293.0101433173, 40.638422, 4.0]], columns=['time', 'rv', 'rv_err'])

barycenter_corr = np.array([9.798384,12.206743,13.575636,
                            14.382747,14.397823,13.634840,
                            12.746271,12.300948,7.496051])

new_rvs['A']['time'] += 2400000.5 - 2455000
new_rvs['B']['time'] += 2400000.5 - 2455000
#new_rvs['A']['rv'] -= 85
new_rvs['A']['rv'] += barycenter_corr
#new_rvs['B']['rv'] -= 85
new_rvs['B']['rv'] += barycenter_corr

#plot.rv_residuals(x, extra_rvs=new_rvs, phased=True)
print(new_rvs['B'])

In [ ]:
plt.plot(nice_units(chain)[...,15].T);

In [ ]:
plt.plot((-2*prob).min(axis=0))
plt.yscale('log')

In [ ]:
plt.plot(-2*prob.T)
plt.yscale('log')

In [ ]:
print(-2*prob[:,-1])

In [ ]:
dof = sum(map(len, plot.ecl_data.values())) + sum(map(len, plot.rv_data.values()))
-2*prob.flatten()[best_indx]/dof

In [ ]:
import corner
corner.corner((nice_units(chain[:, 2000:, :].reshape(-1, 16)).T[chain.reshape(-1, 16).std(axis=0) > 1e-10]).T)

In [ ]:
im = np.degrees(np.arccos(np.sin(chain[:, 3000:, 9])*np.cos(chain[:, 3000:, 10]))).T
plt.hist(im.flatten(), bins=50);

In [ ]:
chain[good_walkers, 1000:, :].reshape(-1, 16).std(axis=0)

In [ ]:
plt.rc('font', size=14)
chain_carmenes = np.load(system + '_chains_carmenes.npy')
prob_carmenes = np.load(system + '_probs_carmenes.npy')
chain_not = np.load(system + '_chains_not.npy')
prob_not = np.load(system + '_probs_not.npy')
fig, axs = plt.subplots(2, 2, figsize=(10,9))
axs = axs.flatten()
def double_hist(ax, par):
    ax.hist(chain_carmenes[...,3000:,par].flatten(), bins=30, density=True, alpha=0.3, label='CARMENES')
    ax.hist(chain_not[...,3000:,par].flatten(), bins=30, density=True, alpha=0.3, label='NOT')
    ax.legend()
double_hist(axs[0], 11)
axs[0].set_xlabel('$M_A$ ($M_\odot$)')
double_hist(axs[1], 12)
axs[1].set_xlabel('$M_B$ ($M_\odot$)')
double_hist(axs[2], 3)
axs[2].set_xlabel('$e_1$')
double_hist(axs[3], 4)
axs[3].set_xlabel('$\omega_1$ (rad)')